In [6]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image_dataset_from_directory

def preprocess_image(image, label):
    image = tf.image.grayscale_to_rgb(image)
    image = image / 255.0
    return image, label

train_ds = image_dataset_from_directory(
    'dataset/train',
    image_size=(64 ,64),
    validation_split=0.2,
    seed=123,
    color_mode='grayscale',
    batch_size=32,
    subset='training'
)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    'dataset/train',
    image_size=(64 ,64),
    validation_split=0.2,
    seed=123,
    color_mode='grayscale',
    batch_size=32,
    subset='validation'
)

train_ds = train_ds.map(lambda x, y: preprocess_image(x, y))
val_ds = val_ds.map(lambda x, y: preprocess_image(x, y))

AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.cache().shuffle(250).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)


Found 28709 files belonging to 7 classes.
Using 22968 files for training.
Found 28709 files belonging to 7 classes.
Using 5741 files for validation.


In [9]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.models import Model

model = VGG16(input_shape=(64, 64, 3), include_top=False, pooling='none', weights='imagenet')

layer_count = 0
for layer in model.layers:
    layer.trainable = False
    layer_count += 1

x = model.output
x = Flatten()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.2)(x)
x = Dense(7, activation='softmax')(x)

model = Model(inputs=model.input, outputs=x)

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 64, 64, 3)]       0         
                                                                 
 block1_conv1 (Conv2D)       (None, 64, 64, 64)        1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 64, 64, 64)        36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 32, 32, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 32, 32, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 32, 32, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 16, 16, 128)       0   

In [11]:
model.fit(
    train_ds,
    epochs=50,
    batch_size=32, 
    validation_data=val_ds
)

Epoch 1/50
718/718 [==============================] - 40s 56ms/step - loss: 1.0494 - accuracy: 0.6063 - val_loss: 1.4966 - val_accuracy: 0.4754
Epoch 2/50
718/718 [==============================] - 40s 56ms/step - loss: 1.0020 - accuracy: 0.6254 - val_loss: 1.4799 - val_accuracy: 0.4766
Epoch 3/50
718/718 [==============================] - 40s 56ms/step - loss: 0.9589 - accuracy: 0.6412 - val_loss: 1.5096 - val_accuracy: 0.4743
Epoch 4/50
718/718 [==============================] - 40s 56ms/step - loss: 0.9121 - accuracy: 0.6633 - val_loss: 1.5793 - val_accuracy: 0.4696
Epoch 5/50
718/718 [==============================] - 40s 56ms/step - loss: 0.8736 - accuracy: 0.6714 - val_loss: 1.5678 - val_accuracy: 0.4804
Epoch 6/50
718/718 [==============================] - 40s 56ms/step - loss: 0.8337 - accuracy: 0.6899 - val_loss: 1.6036 - val_accuracy: 0.4738
Epoch 7/50
718/718 [==============================] - 41s 57ms/step - loss: 0.7860 - accuracy: 0.7092 - val_loss: 1.5988 - val_accuracy:

In [12]:
i = 13
count = 0
for layer in model.layers:
    if i == count:
        layer.trainable = True
        i += 1
    count += 1

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 64, 64, 3)]       0         
                                                                 
 block1_conv1 (Conv2D)       (None, 64, 64, 64)        1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 64, 64, 64)        36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 32, 32, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 32, 32, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 32, 32, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 16, 16, 128)       0   

In [13]:
model.fit(
    train_ds,
    epochs=50,
    batch_size=32,
    validation_data=val_ds,
)

Epoch 1/50
718/718 [==============================] - 73s 100ms/step - loss: 2.9846 - accuracy: 0.2878 - val_loss: 1.5162 - val_accuracy: 0.4102
Epoch 2/50
718/718 [==============================] - 70s 97ms/step - loss: 1.4401 - accuracy: 0.4428 - val_loss: 1.3794 - val_accuracy: 0.4609
Epoch 3/50
718/718 [==============================] - 71s 99ms/step - loss: 1.3556 - accuracy: 0.4856 - val_loss: 1.3254 - val_accuracy: 0.5003
Epoch 4/50
718/718 [==============================] - 70s 98ms/step - loss: 1.2730 - accuracy: 0.5199 - val_loss: 1.2826 - val_accuracy: 0.5189
Epoch 5/50
718/718 [==============================] - 64s 89ms/step - loss: 1.2298 - accuracy: 0.5379 - val_loss: 1.3648 - val_accuracy: 0.5118
Epoch 6/50
718/718 [==============================] - 64s 88ms/step - loss: 1.1868 - accuracy: 0.5598 - val_loss: 1.2853 - val_accuracy: 0.5285
Epoch 7/50
718/718 [==============================] - 64s 89ms/step - loss: 1.1643 - accuracy: 0.5646 - val_loss: 1.2420 - val_accuracy

In [14]:
test_ds = tf.keras.preprocessing.image_dataset_from_directory(
     'dataset/test',
    image_size=(64 ,),
    color_mode='grayscale',
    batch_size=32,
)

test_ds = test_ds.map(lambda x, y: preprocess_image(x, y))

AUTOTUNE = tf.data.AUTOTUNE
test_ds = test_ds.cache().shuffle(250).prefetch(buffer_size=AUTOTUNE)

test_loss, test_accuracy = model.evaluate(test_ds)

print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")

Found 7178 files belonging to 7 classes.


InvalidArgumentError: Graph execution error:

Detected at node 'Equal' defined at (most recent call last):
    File "C:\Users\dindo\anaconda3\envs\testenv\lib\runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "C:\Users\dindo\anaconda3\envs\testenv\lib\runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "C:\Users\dindo\anaconda3\envs\testenv\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "C:\Users\dindo\anaconda3\envs\testenv\lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
      app.start()
    File "C:\Users\dindo\anaconda3\envs\testenv\lib\site-packages\ipykernel\kernelapp.py", line 701, in start
      self.io_loop.start()
    File "C:\Users\dindo\anaconda3\envs\testenv\lib\site-packages\tornado\platform\asyncio.py", line 205, in start
      self.asyncio_loop.run_forever()
    File "C:\Users\dindo\anaconda3\envs\testenv\lib\